In [1]:
import pandas as pd
import json
import numpy as np

In [6]:
df: pd.DataFrame = pd.read_excel(
    "data\\Agribalyse 2023 3.1 with WiseFins categorisation.xlsx"
)

df.head()

,Code\nAGB,WiseFins FR Category,WiseFins EN Category,WiseFins FR Subcategory,WiseFins EN Subcategory,Code\nCIQUAL,Groupe d'aliment,Sous-groupe d'aliment,Nom du Produit en Français,LCI Name,...,Eutrophisation terreste (mol N eq/kg de produit),Eutrophisation eaux douces (E-03 kg P eq/kg de produit),Eutrophisation marine (E-03 kg N eq/kg de produit),Écotoxicité pour écosystèmes aquatiques d'eau douce (CTUe/kg de produit),Utilisation du sol (Pt/kg de produit),Épuisement des ressources eau (m3 depriv./kg de produit),Épuisement des ressources énergétiques (MJ/kg de produit),Épuisement des ressources minéraux (E-06 kg Sb eq/kg de produit),Effets toxicologiques sur la santé humaine : substances non-cancérogènes (CTUh/kg de produit),Effets toxicologiques sur la santé humaine : substances cancérogènes (CTUh/kg de produit)
0,11084,épicerie,dry storage,algues,algae,11084,aides culinaires et ingrédients divers,algues,"Agar (algue), cru","Seaweed, agar, raw",...,0.127350,0.001868,0.013280,43.368466,24.699395,3.079121,315.057170,0.000088,4.771949e-08,6.108019e-09
1,11023,épicerie,dry storage,herbes,herbs,11023,aides culinaires et ingrédients divers,herbes,"Ail séché, poudre","Garlic, powder, dried",...,0.012785,0.000125,0.002379,5.672531,20.508995,2.585763,12.701604,0.000007,9.281982e-09,4.343632e-10
2,11000,épicerie,dry storage,herbes,herbs,11000,aides culinaires et ingrédients divers,herbes,"Ail, cru","Garlic, fresh",...,0.006394,0.000069,0.002197,4.064650,18.871203,3.104146,6.275385,0.000004,6.066248e-09,2.824785e-10
3,11093,épicerie,dry storage,herbes,herbs,11093,aides culinaires et ingrédients divers,herbes,"Aneth, frais","Dill, fresh",...,0.031181,0.000203,0.004050,5.534832,36.288833,5.290335,12.553109,0.000005,9.739990e-09,5.867577e-10
4,20995,épicerie,dry storage,algues,algae,20995,aides culinaires et ingrédients divers,algues,"Ao-nori (Enteromorpha sp.), séchée ou déshydratée","Sea lettuce (Enteromorpha sp.), dried or dehyd...",...,0.127350,0.001868,0.013280,43.368466,24.699395,3.079121,315.057170,0.000088,4.771949e-08,6.108019e-09


In [16]:
def read_and_clean_xlsx(file_path):
    # Read all sheets from the spreadsheet; this returns a dictionary
    all_sheets_data = pd.read_excel(file_path, sheet_name=None)

    # Initialize a dictionary to hold the cleaned data
    cleaned_data = {}

    # Iterate over each sheet in the file
    for sheet_name, data in all_sheets_data.items():
        # Drop rows where all values are NaN
        cleaned_rows = data.dropna(how='all')
        # Drop columns where all values are NaN
        cleaned_columns = cleaned_rows.dropna(axis=1, how='all')
        # Store the cleaned DataFrame in the dictionary
        cleaned_data[sheet_name] = cleaned_columns

    return cleaned_data


In [19]:

# Replace 'your_file_path.xlsx' with the path to your .xlsx file
file_path = "data\\20231207_client_data\\recipe manual input on excel\\Hotel 2\\chef_manual_recipe_hotel2  .xlsx"
cleaned_data = read_and_clean_xlsx(file_path)

# Example: Print the cleaned data for the first sheet
first_sheet_name = list(cleaned_data.keys())[0]
print(f"Data from the first sheet '{first_sheet_name}':")
print(cleaned_data[first_sheet_name].head())

Data from the first sheet 'Tiramisu   ':
   Unnamed: 1 PLATE SPECIFICATION / รายละเอียด  Unnamed: 3 Unnamed: 4  \
20        NaN                      Tiramisu            NaN        NaN   
21        NaN                     Ingredients:         NaN        NaN   
22    432.827                      Tiramisu           0.18        465   
23        NaN                  chocholate ball        0.01       2000   
24     DZ0001                       extra cost        1.00          5   

    Unnamed: 5  
20         NaN  
21         NaN  
22        83.7  
23        20.0  
24         5.0  


In [4]:
def parse_sheet_to_json(df):
    # Initialize the final JSON structure
    json_data = []

    # Find the rows which contain the summary information like "cost", "SP in THB", and "Food cost %"
    # Look for the last occurrence of these key terms
    summary_indices = {
        'cost': None,
        'SP in THB': None,
        'Food cost %': None
    }
    for key in summary_indices.keys():
        found_indices = df.index[df.iloc[:,0] == key].tolist()
        if found_indices:
            summary_indices[key] = max(found_indices)  # Get the last occurrence
    
    # Initialize the recipe dictionary with summary information
    recipe_dict = {
        "Recette": df.iloc[0,0],  # Assuming the first non-empty cell in the sheet is the recipe name
    }
    
    # Add summary information if available
    for key, index in summary_indices.items():
        if index is not None:
            recipe_dict[key] = df.iloc[index, 1] if len(df.columns) > 1 else None
    
    # Add ingredients list
    recipe_dict["Ingrédients"] = []
    
    # Assuming that the ingredients list starts after the recipe name
    # and ends before the summary or at the end of the sheet if no summary is found
    start_index = 1  # Start after the recipe name
    end_index = min(filter(None, summary_indices.values())) if any(summary_indices.values()) else len(df)
    
    # Parse the ingredients data
    for _, row in df.iloc[start_index:end_index].iterrows():
        if not row.isnull().all():  # Skip completely empty rows
            ingredient_dict = {
                "code": row[0],
                "Nom": row[1],
                "Quantité": row[2],
                "Prix": row[3],
                "Cout": row[4] if len(row) > 4 else None
            }
            recipe_dict["Ingrédients"].append(ingredient_dict)
    
    json_data.append(recipe_dict)
    return json.dumps(json_data, indent=4, ensure_ascii=False)


In [6]:
# Applying the parse_sheet_to_json function to all sheets in the Excel file.
file_path = "data\\20231207_client_data\\recipe manual input on excel\\Hotel 2\\chef_manual_recipe_hotel2  .xlsx"

# First, we need to reload all the sheets because we have modified the dataframes previously.
all_sheets_data = pd.read_excel(file_path, sheet_name=None)

# Now we will apply the function to each sheet and collect the JSON strings
all_sheets_json = {}
for sheet_name, data in all_sheets_data.items():
    # Clean the data first by dropping empty rows and columns
    data_cleaned = data.dropna(how='all').dropna(axis=1, how='all')
    # Parse the cleaned data to JSON
    all_sheets_json[sheet_name] = parse_sheet_to_json(data_cleaned)

# Combine all the JSON strings into one list
combined_json_list = '[' + ','.join(all_sheets_json.values()) + ']'

# Convert the combined string back to a JSON object to pretty print and ensure it is correctly formatted
combined_json = json.loads(combined_json_list)
pretty_combined_json = json.dumps(combined_json, indent=4, ensure_ascii=False)
pretty_combined_json


'[\n    [\n        {\n            "Recette": NaN,\n            "Ingrédients": [\n                {\n                    "code": NaN,\n                    "Nom": "Ingredients:",\n                    "Quantité": NaN,\n                    "Prix": NaN,\n                    "Cout": NaN\n                },\n                {\n                    "code": 432.827,\n                    "Nom": "Tiramisu   ",\n                    "Quantité": 0.18,\n                    "Prix": 465,\n                    "Cout": 83.7\n                },\n                {\n                    "code": NaN,\n                    "Nom": "chocholate ball",\n                    "Quantité": 0.01,\n                    "Prix": 2000,\n                    "Cout": 20.0\n                },\n                {\n                    "code": "DZ0001",\n                    "Nom": "extra cost",\n                    "Quantité": 1.0,\n                    "Prix": 5,\n                    "Cout": 5.0\n                },\n                {\n

In [6]:
import pandas as pd

# Provided JSON data
json_data = {
    "name": "F10000009   Mixed Leaf Salad",
    "total": {
        "Sell price": "64.36",
        "Sell": "290.00",
        "Cost": "1900-01-22 02:24:00",
        "Cost%": "%"
    },
    "ingredients": [
        {
            "code": "102.455",
            "name": "Lettuce Mixed AF@@@",
            "L": 1.0,
            "S": 0.0,
            "Qty": "120.000",
            "Unit": "GRAM",
            "Sell_price": "455.50",
            "Cost%": "54.66"
        },
        {
            "code": "150.736",
            "name": "Tomato Teardrop/Cherry/",
            "L": 1.0,
            "S": 0.0,
            "Qty": "20.000",
            "Unit": "GRAM",
            "Sell_price": "52.00",
            "Cost%": "1.04"
        },
        # ... other ingredients ...
    ]
}

# Initialize an empty list to store the DataFrame rows
rows = []

# Loop through each ingredient and create a row for each
for ingredient in json_data['ingredients']:
    # Calculate the cost by dividing the sell price by the cost percentage
    # and converting the result to a float
    sell_price = float(ingredient['Sell_price'])
    cost_percentage = float(ingredient['Cost%']) / 100 
    cost = sell_price * cost_percentage
    
    # Create a dictionary for the row
    row = {
        'resto': 'iscala',
        'subRecipe': None,
        'recipeGroup': 'food' if json_data['name'][0] == 'F' else 'beverage',
        'recipeGroupParent': None,
        'recipeName': json_data['name'],
        'recipeCompose': ingredient['name'],
        'AGBCode': None,
        'Quantity': ingredient['Qty'],
        'cost': cost,
        'InventoryCode': ingredient['code'],
        'unit': ingredient['Unit'],
        'quantityAfter': None,
        'portion': 1,
        'statut': None,
        'photo': None,
        'selling': sell_price,
        'category': None,
        'subcategory': None
    }
    
    # Append the row dictionary to the rows list
    rows.append(row)

# Convert the list of rows to a DataFrame
df = pd.DataFrame(rows)

# Display the DataFrame
df.head()


0.5466
0.0104


,resto,subRecipe,recipeGroup,recipeGroupParent,recipeName,recipeCompose,AGBCode,Quantity,cost,InventoryCode,unit,quantityAfter,portion,statut,photo,selling,category,subcategory
0,iscala,None,food,None,F10000009 Mixed Leaf Salad,Lettuce Mixed AF@@@,None,120.000,248.9763,102.455,GRAM,None,1,None,None,455.5,None,None
1,iscala,None,food,None,F10000009 Mixed Leaf Salad,Tomato Teardrop/Cherry/,None,20.000,0.5408,150.736,GRAM,None,1,None,None,52.0,None,None


In [16]:
import json
import pandas as pd

def convert_price_to_float(price_str):
    # Supprime les virgules et convertit en float
    return float(price_str.replace(',', ''))

def create_dataframe_from_json_file(json_file_path):
    # Load JSON data from the file
    with open(json_file_path, 'r') as file:
        json_data = json.load(file)

    # Initialize an empty list to store the DataFrame rows
    rows = []

    # Loop through each entry in the JSON data and create a row for each ingredient
    for item in json_data:
        # Parse the item name to determine the recipe group
        first_letter = item['name'].strip()[0].upper()
        recipe_group = 'food' if first_letter == 'F' else 'beverage' if first_letter == 'B' else 'other'

        for ingredient in item['ingredients']:

            try:
                sell_price = convert_price_to_float(ingredient['Sell_price'].replace(',', '').replace('R', ''))
            except ValueError:
                sell_price = convert_price_to_float(ingredient['Cost%'].strip('%')) if ingredient['Cost%'].strip('%') else 0

            # Calculate cost if possible
            cost_percentage = convert_price_to_float(ingredient['Cost%'].strip('%')) / 100 if ingredient['Cost%'].strip('%') else 0
            cost = sell_price if cost_percentage == 0 else sell_price * cost_percentage

            # sell_price = convert_price_to_float(ingredient['Sell_price'])
            # cost_percentage = convert_price_to_float(ingredient['Cost%'].strip('%')) / 100 if ingredient['Cost%'].strip('%') else 1
            # cost = sell_price * cost_percentage if cost_percentage else 0

            # Create a dictionary for the row
            row = {
                'resto': 'iscala',
                'subRecipe': '',
                'recipeGroup': recipe_group,
                'recipeGroupParent': '',
                'recipeName': item['name'].strip(),
                'recipeCompose': ingredient['name'].strip(),
                'AGBCode': '',
                'Quantity': ingredient['Qty'].strip(),
                'cost': cost,
                'InventoryCode': ingredient['code'].strip(),
                'unit': ingredient['Unit'].strip(),
                'quantityAfter': '',
                'portion': 1,
                'statut': '',
                'photo': '',
                'selling': ingredient['Sell_price'].strip(),
                'category': '',
                'subcategory': ''
            }

            # Append the row dictionary to the rows list
            rows.append(row)

    # Convert the list of rows to a DataFrame
    return pd.DataFrame(rows)

# Replace '/path/to/your/file.json' with the actual path to your JSON file
df = create_dataframe_from_json_file('data_converted\\iscala\\beverage_recipe_iscala.json')
print(df.head())


    resto subRecipe recipeGroup recipeGroupParent                  recipeName  \
0  iscala              beverage                               B000400100 Coke   
1  iscala              beverage                             B000400101 Sprite   
2  iscala              beverage                    B000400102 Schweppes tonic   
3  iscala              beverage                     B000400103 Schweppes soda   
4  iscala              beverage                              B000400104 Water   

            recipeCompose AGBCode Quantity      cost InventoryCode       unit  \
0             Coke Can MB            1.000  1.836025       284.162     BOTTLE   
1           Sprite Can MB            1.000  1.836025       284.183  MILILITER   
2  Schweppes Tonic Can MB            1.000  1.710864       284.165       EACH   
3   Schweppes Soda Can MB            1.000  1.710864       284.164       EACH   
4   Water Drinking Singha          500.000  0.038025       284.262  MILILITER   

  quantityAfter  portion s

In [48]:
df: pd.DataFrame = pd.read_excel(
    "data\\20231207_client_data\\export from Purchasing software\\ISCALA (Hotel 2)\\food_recipe_iscala.xlsx",
    sheet_name='Sheet1',
    skiprows=6,
    header=None
)

df.head()


,1,2,3,5,6,7,9,10,12,13,...,34,35,37,38,39,40,41,43,44,45
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0000,9.000000
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.3050,14.386667
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.9082,19.295251
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.2700,0.020000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.1175,0.030000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.6000,0.140000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.7875,22.400000
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.7500,48.750000


In [50]:
df

,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,42,43,44,45
0,Template : F10000001 DK Caesar Salad,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,102.462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lettuce Romaine AF,NaN,...,237.50,NaN,NaN,NaN,NaN,NaN,59.38,NaN,NaN,NaN
2,134.725,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cheese Parmesan 14/16,NaN,...,689.00,NaN,NaN,NaN,NaN,NaN,20.67,NaN,NaN,NaN
3,136.855,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pepper Black/Whit Crack,NaN,...,300.00,NaN,NaN,NaN,NaN,NaN,0.30,NaN,NaN,NaN
4,414.304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bacon Crisps,NaN,...,"14,080.00",NaN,NaN,NaN,NaN,NaN,14.08,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42331,DZ0001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Extra Cost,NaN,...,11.20,NaN,NaN,NaN,NaN,NaN,11.20,NaN,NaN,NaN
42332,412.952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Juice Watermelon Frs,NaN,...,"13,440.00",NaN,NaN,NaN,NaN,NaN,13.44,NaN,NaN,NaN
42333,414.780,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Smoothie Banana,NaN,...,"16,942.40",NaN,NaN,NaN,NaN,NaN,17.40,NaN,NaN,NaN
42334,- - - - - - - - - - - - - - - - - - - - - - - ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
def convert_date_or_skip(value):
    if pd.isnull(value):  # Vérifie si la valeur est NaN
        return np.nan
    elif isinstance(value, pd.Timestamp):  # Si la valeur est un Timestamp, convertissez-la en nombre
        return float(value.timestamp())
    else:  # Sinon, laissez la valeur telle quelle (pour les chaînes de caractères)
        return value

In [51]:
df = df.dropna(axis = 0, how = 'all')
df = df[~df[0].isin(['- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -'])]

In [48]:
list_products: list[dict] = []
current_product: dict = {}
list_ingredients: list[dict] = []

for index, row in df.iterrows():
    # First item row
    if str(row[0]).find('Template : ') != -1:
        current_product = {
            "name": row[0][11:]
        }

        continue
    
    # Last item row
    if str(row[0]) == 'Total for Stock Item':
        current_product['total'] : dict = {
            # TODO rename
            'total_first_col': row[11],
            'total_second_col': row[17],
            'total_third_col': str(row[21]),
            'total_fourth_col': row[24]
            
        }

        current_product['ingredients'] = list_ingredients
        list_products.append(current_product)
        
        list_ingredients = []

        continue
        
    # Intermediate row (Ingredient)
    list_ingredients.append({
        'code': row[0],
        'name': row[8],
        
        # TODO rename those columns
        'L': row[14],
        'S': row[20],
        'Qty': row[24],
        'Unit': row[32],
        'Sell_price': row[36],
        'Cost%': row[42]
    })


In [52]:
list_products = []
current_product = {}  
list_ingredients = []  

for index, row in df.iterrows():
    # First item row
    if pd.notnull(row[0]) and str(row[0]).startswith('Template : '):
        if current_product.get("name"):  
            current_product['ingredients'] = list_ingredients
            list_products.append(current_product)
            list_ingredients = []  
        current_product = {
            "name": row[0][11:]
        }
        continue
    
    # Last item row
    if pd.notnull(row[0]) and str(row[0]) == 'Total for Stock Item':
        current_product['total'] = {
            'Sell price': row[11],
            'Sell': row[17],
            'Cout': str(row[21]),
            'Cout%': row[24]
        }
        current_product['ingredients'] = list_ingredients
        list_products.append(current_product)
        current_product = {}  
        list_ingredients = []  
        continue
    
    # Intermediate row (Ingredient)
    # code = row[0] if pd.notnull(row[0]) else '000'
    # name = row[8] if pd.notnull(row[8]) else row[0]
    
    list_ingredients.append({
        'code': row[0] if pd.notnull(row[0]) else '000',
        'name': row[8] if pd.notnull(row[8]) else row[0],
        'L': row[14] if pd.notnull(row[14]) else '000',
        'S': row[20] if pd.notnull(row[20]) else '000',
        'Qty': row[24] if pd.notnull(row[24]) else '000',
        'Unit': row[32] if pd.notnull(row[32]) else '000',
        'Sell_price': row[36] if pd.notnull(row[36]) else '000',
        'Cost%': row[42] if pd.notnull(row[42]) else '000'
    })


if current_product.get("name") and (not list_products or current_product != list_products[-1]):
    current_product['ingredients'] = list_ingredients
    list_products.append(current_product)


In [53]:
with open("data_converted\\iscala\\food_recipe_iscala.json", "w") as outfile:
    # json_data refers to the above JSON
    json.dump(list_products, outfile, indent=4)

In [2]:
df: pd.DataFrame = pd.read_excel(
    "data\\20231207_client_data\\recipe manual input on excel\Hotel 1\\chef_manual_recipe_pizza_hotel1.xlsx",
    sheet_name='Pizza Puttanesca',
    header=None
)

df

,0,1,2,3,4,5,6,7
0,Hotel 1,NaN,NaN,NaN,NaN,Chef name,NaN,NaN
1,Pizza Puttanesca,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Date:,NaN,NaN,NaN,Number of:,Portion,1,NaN
3,Change:,NaN,NaN,NaN,Cost per portion:,NaN,2.614972,NaN
4,Kind of product :,NaN,NaN,NaN,Selling price:,Tax excluding:,28,NaN
5,Outlet Name:,NaN,NaN,NaN,NaN,Tax including:,33.32,NaN
6,NaN,NaN,NaN,NaN,Cost :,NaN,0.093392,NaN
7,Name of ingredient:,NaN,Birchstreet,Quantity,Quantity2,Unit,Cost per unit,Cost per ingredient
8,Pizza Puttanesca,NaN,NaN,NaN,1,NaN,NaN,NaN
9,PIZZA DOUGH,NaN,NaN,0.26,3.42,Kg,11.5618,0.878967


In [16]:
import numpy as np

In [3]:
name_pizza = df.iloc[1, 0]  # Name seems to be in the second row, first column

cout_portion_pizza = df.iloc[3, 6]  # "Cout Portion" seems to be in the fourth row, seventh column
sell_price_without_tax_pizza = df.iloc[4, 6]  # "Sell Price without Tax" in the fifth row, seventh column
sell_price_with_tax_pizza = df.iloc[5, 6]  # "Sell Price with Tax" in the sixth row, seventh column
cost_percent_pizza = df.iloc[6, 6]  # "Cost%" in the seventh row, seventh column
    
extracted_values_pizza = {

    "Name": name_pizza,

    "Cout Portion": cout_portion_pizza,

    "Sell Price without Tax": sell_price_without_tax_pizza,

    "Sell Price with Tax": sell_price_with_tax_pizza,

    "Cost%": cost_percent_pizza,

    "Ingredient": []

}

    
start_row_index = 9



# Initialize an empty list to store the extracted information

extracted_info = []



# Iterate through the DataFrame rows starting from the specified row

for row in df.iloc[start_row_index:].itertuples(index=False):

    # Check if "direction" is in any cell of the current row

    if any("direction" in str(cell).lower() for cell in row):

        break  # Stop the loop if "direction" is found

    else:

        extracted_info.append(row)  # Add the row to the list if "direction" is not found


    formatted_data = []

for row in extracted_info:

# Skip rows where the first cell (which should contain the name) is None or empty

    if row[0] is None or str(row[0]).strip() == '':

        continue

    formatted_row = {

        "code": row[2],

        "Nom": row[0],

        "Quantity": row[3],

        "Quantity2": row[4],

        "Unit": row[5],

        "Cost per unit": row[6],

        "Cost per ingredient": row[7]

    }
    formatted_data.append(formatted_row)

for item in formatted_data:

    # Convert 'code' NaN to actual NaN, and evaluate 'Quantity2', 'Cost per unit', and 'Cost per ingredient'

    item['code'] = np.nan if item['code'] is None else item['code']

    # Assuming 'Quantity2', 'Cost per unit', and 'Cost per ingredient' are correctly calculated, we directly use them here

    extracted_values_pizza['Ingredient'].append(item)



extracted_values_pizza

{'Name': 'Pizza Puttanesca',
 'Cout Portion': 2.61497197,
 'Sell Price without Tax': 28,
 'Sell Price with Tax': 33.32,
 'Cost%': 0.09339185608,
 'Ingredient': [{'code': nan,
   'Nom': 'PIZZA DOUGH',
   'Quantity': 0.26,
   'Quantity2': 3.42,
   'Unit': 'Kg',
   'Cost per unit': 11.5618,
   'Cost per ingredient': 0.8789672515},
  {'code': 130.461,
   'Nom': 'Flour caputo 00 Pizza',
   'Quantity': 2,
   'Quantity2': 2,
   'Unit': 'Kg',
   'Cost per unit': 5.5,
   'Cost per ingredient': 11},
  {'code': nan,
   'Nom': 'Water',
   'Quantity': 1.3,
   'Quantity2': 1.3,
   'Unit': 'Kg',
   'Cost per unit': 0,
   'Cost per ingredient': 0},
  {'code': 137.937,
   'Nom': 'Active dry yeast',
   'Quantity': 0.01,
   'Quantity2': 0.01,
   'Unit': 'Kg',
   'Cost per unit': 5.24,
   'Cost per ingredient': 0.0524},
  {'code': 130.406,
   'Nom': 'Kosha Salt',
   'Quantity': 0.06,
   'Quantity2': 0.06,
   'Unit': 'Kg',
   'Cost per unit': 5.24,
   'Cost per ingredient': 0.3144},
  {'code': 131.202,
   

In [4]:
import pandas as pd
import numpy as np

# Chemin du fichier Excel
file_path = "data/20231207_client_data/recipe manual input on excel/Hotel 1/chef_manual_recipe_conversion_hotel1.xlsx"

# Charger le fichier Excel pour obtenir les noms de toutes les feuilles
xls = pd.ExcelFile(file_path)

# Obtenir tous les noms des feuilles, puis supprimer le premier (la première feuille)
sheet_names = xls.sheet_names[1:]  # Prendre toutes les feuilles sauf la première

# Liste pour stocker les données extraites de chaque feuille
all_extracted_data = []

# Boucle pour lire chaque feuille et extraire les données
for sheet_name in sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet_name, header=None)
    
    # Extraction des informations spécifiques de la feuille
    name_pizza = df.iloc[1, 0]
    cout_portion_pizza = df.iloc[3, 6]
    sell_price_without_tax_pizza = df.iloc[4, 6]
    sell_price_with_tax_pizza = df.iloc[5, 6]
    cost_percent_pizza = df.iloc[6, 6]
    
    extracted_values_pizza = {
        "Name": name_pizza,
        "Cout Portion": cout_portion_pizza,
        "Sell Price without Tax": sell_price_without_tax_pizza if pd.notnull(sell_price_without_tax_pizza) else '000',
        "Sell Price with Tax": sell_price_with_tax_pizza if pd.notnull(sell_price_with_tax_pizza) else '000',
        "Cost%": cost_percent_pizza if pd.notnull(cost_percent_pizza) else '000',
        "Ingredient": []
    }
    
    start_row_index = 9
    extracted_info = []
    
    for row in df.iloc[start_row_index:].itertuples(index=False):
        if any("direction" in str(cell).lower() for cell in row):
            break
        else:
            extracted_info.append(row)
    
    formatted_data = []
    for row in extracted_info:
        if row[0] is None or str(row[0]).strip() == '':
            continue
        formatted_row = {
            "code": row[2] if pd.notnull(row[2]) else '000',
            "Nom": row[0] if pd.notnull(row[0]) else '000',
            "Quantity": row[3] if pd.notnull(row[3]) else '000',
            "Quantity2": row[4] if pd.notnull(row[4]) else '000',
            "Unit": row[5] if pd.notnull(row[5]) else '000',
            "Cost per unit": row[6] if pd.notnull(row[6]) else '000',
            "Cost per ingredient": row[7] if pd.notnull(row[7]) else '000'
        }
        formatted_data.append(formatted_row)
    
    for item in formatted_data:
        extracted_values_pizza['Ingredient'].append(item)
    
    # Ajouter les données extraites de la feuille actuelle à la liste globale
    all_extracted_data.append(extracted_values_pizza)

# 'all_extracted_data' contient maintenant les données extraites de chaque feuille à l'exception de la première


In [5]:
with open("data_converted\\manual\\chef_manual_recipe_conversion_hotel1.json", "w") as outfile:
    # json_data refers to the above JSON
    json.dump(all_extracted_data, outfile, indent=4)

In [64]:
all_extracted_data

[{'Name': 'Sample',
  'Cout Portion': 0,
  'Sell Price without Tax': nan,
  'Sell Price with Tax': 0,
  'Cost%': '#DIV/0!',
  'Ingredient': [{'code': '000',
    'Nom': '000',
    'Quantity': '000',
    'Quantity2': 0,
    'Unit': '000',
    'Cost per unit': '000',
    'Cost per ingredient': 0},
   {'code': '000',
    'Nom': '000',
    'Quantity': '000',
    'Quantity2': 0,
    'Unit': '000',
    'Cost per unit': '000',
    'Cost per ingredient': 0},
   {'code': '000',
    'Nom': '000',
    'Quantity': '000',
    'Quantity2': 0,
    'Unit': '000',
    'Cost per unit': '000',
    'Cost per ingredient': 0},
   {'code': '000',
    'Nom': '000',
    'Quantity': '000',
    'Quantity2': 0,
    'Unit': '000',
    'Cost per unit': '000',
    'Cost per ingredient': 0},
   {'code': '000',
    'Nom': '000',
    'Quantity': '000',
    'Quantity2': 0,
    'Unit': '000',
    'Cost per unit': '000',
    'Cost per ingredient': 0},
   {'code': '000',
    'Nom': '000',
    'Quantity': '000',
    'Quantit

In [15]:
extracted_values_pizza

{'Name': 'Pizza Puttanesca',
 'Cout Portion': 2.61497197,
 'Sell Price without Tax': 28,
 'Sell Price with Tax': 33.32,
 'Cost%': 0.09339185608}

In [14]:

formatted_data

[{'code': nan,
  'Nom': 'PIZZA DOUGH',
  'Quantity': 0.26,
  'Quantity2': 3.42,
  'Unit': 'Kg',
  'Cost per unit': 11.5618,
  'Cost per ingredient': 0.8789672515},
 {'code': 130.461,
  'Nom': 'Flour caputo 00 Pizza',
  'Quantity': 2,
  'Quantity2': 2,
  'Unit': 'Kg',
  'Cost per unit': 5.5,
  'Cost per ingredient': 11},
 {'code': nan,
  'Nom': 'Water',
  'Quantity': 1.3,
  'Quantity2': 1.3,
  'Unit': 'Kg',
  'Cost per unit': 0,
  'Cost per ingredient': 0},
 {'code': 137.937,
  'Nom': 'Active dry yeast',
  'Quantity': 0.01,
  'Quantity2': 0.01,
  'Unit': 'Kg',
  'Cost per unit': 5.24,
  'Cost per ingredient': 0.0524},
 {'code': 130.406,
  'Nom': 'Kosha Salt',
  'Quantity': 0.06,
  'Quantity2': 0.06,
  'Unit': 'Kg',
  'Cost per unit': 5.24,
  'Cost per ingredient': 0.3144},
 {'code': 131.202,
  'Nom': 'Honey',
  'Quantity': 0.05,
  'Quantity2': 0.05,
  'Unit': 'Kg',
  'Cost per unit': 3.9,
  'Cost per ingredient': 0.195},
 {'code': nan,
  'Nom': 'TOMATO SAUCE',
  'Quantity': 0.08,
  'Qua

In [9]:
import openpyxl
from openpyxl import load_workbook
def read_all_data(path):
    workbook = load_workbook(filename=path)
    for sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]
        print(f"Title = {sheet.title}")
        for row in sheet.rows:
            for cell in row:
                if isinstance(cell, openpyxl.cell.cell.MergedCell):
                    # Skip this cell
                    continue
                print(f"{cell.column_letter}{cell.row} = {cell.value}")
if __name__ == "__main__":
    read_all_data("data\\20231207_client_data\\recipe manual input on excel\\Hotel 2\\chef_manual_recipe_hotel2  .xlsx")

Title = Tiramisu   
A1 = None
B1 = None
C1 = PLATE SPECIFICATION / รายละเอียด
D1 = None
E1 = None
F1 = None
A2 = None
B2 = None
C2 = None
D2 = None
E2 = None
F2 = None
A3 = None
B3 = None
C3 = None
D3 = None
E3 = None
F3 = None
A4 = None
B4 = None
C4 = None
D4 = None
E4 = None
F4 = None
A5 = None
B5 = None
C5 = None
D5 = None
E5 = None
F5 = None
A6 = None
B6 = None
C6 = None
D6 = None
E6 = None
F6 = None
A7 = None
B7 = None
C7 = None
D7 = None
E7 = None
F7 = None
A8 = None
B8 = None
C8 = None
D8 = None
E8 = None
F8 = None
A9 = None
B9 = None
C9 = None
D9 = None
E9 = None
F9 = None
A10 = None
B10 = None
C10 = None
D10 = None
E10 = None
F10 = None
A11 = None
B11 = None
C11 = None
D11 = None
E11 = None
F11 = None
A12 = None
B12 = None
C12 = None
D12 = None
E12 = None
F12 = None
A13 = None
B13 = None
C13 = None
D13 = None
E13 = None
F13 = None
A14 = None
B14 = None
C14 = None
D14 = None
E14 = None
F14 = None
A15 = None
B15 = None
C15 = None
D15 = None
E15 = None
F15 = None
A16 = None
B16 =

In [10]:
from openpyxl import load_workbook
def read_all_data(path):
    
    workbook = load_workbook(filename=path)
    for sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]
        print(f"Title = {sheet.title}")
        for value in sheet.iter_rows(values_only=True):
            print(value)
if __name__ == "__main__":
    read_all_data("data\\20231207_client_data\\recipe manual input on excel\\Hotel 2\\chef_manual_recipe_hotel2  .xlsx")

Title = Tiramisu   
(None, None, 'PLATE SPECIFICATION / รายละเอียด', None, None, None)
(None, None, None, None, None, None)
(None, None, None, None, None, None)
(None, None, None, None, None, None)
(None, None, None, None, None, None)
(None, None, None, None, None, None)
(None, None, None, None, None, None)
(None, None, None, None, None, None)
(None, None, None, None, None, None)
(None, None, None, None, None, None)
(None, None, None, None, None, None)
(None, None, None, None, None, None)
(None, None, None, None, None, None)
(None, None, None, None, None, None)
(None, None, None, None, None, None)
(None, None, None, None, None, None)
(None, None, None, None, None, None)
(None, None, None, None, None, None)
(None, None, None, None, None, None)
(None, None, None, None, None, None)
(None, None, None, None, None, None)
(None, None, 'Tiramisu   ', None, None, None)
(None, None, 'Ingredients:', None, None, None)
(None, 432.827, 'Tiramisu   ', 0.18, 465, '=E24*D24')
(None, None, 'chocholate b

In [5]:
df_test_hotel2: pd.DataFrame = pd.read_excel(
    "data\\20231207_client_data\\recipe manual input on excel\\Hotel 2\\chef_manual_recipe_hotel2  .xlsx",
    sheet_name='Cappuccino',
    skiprows=21,
    header=None
)

df_test_hotel2

,0,1,2,3,4,5,6
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Cappuccino,NaN,NaN,NaN,NaN
3,NaN,NaN,Ingredients:,NaN,NaN,NaN,NaN
4,NaN,134.637,Cappuccino,0.100,475,47.50000,NaN
5,NaN,433.222,chocholate,0.005,473,2.36500,NaN
6,NaN,NaN,NaN,NaN,NaN,0.00000,NaN
7,NaN,DZ0001,extra cost,1.000,5,5.00000,NaN
8,NaN,NaN,NaN,NaN,NaN,0.00000,NaN
9,NaN,NaN,NaN,NaN,cost,250.00000,54.9


In [9]:
df_test_hotel2 = df_test_hotel2.dropna(axis = 0, how = 'all')
df_test_hotel2 = df_test_hotel2.dropna(axis = 1, how = 'all')


In [10]:
df_test_hotel2

,1,2,3,4,5,6
2,NaN,Cappuccino,NaN,NaN,NaN,NaN
3,NaN,Ingredients:,NaN,NaN,NaN,NaN
4,134.637,Cappuccino,0.100,475,47.50000,NaN
5,433.222,chocholate,0.005,473,2.36500,NaN
6,NaN,NaN,NaN,NaN,0.00000,NaN
7,DZ0001,extra cost,1.000,5,5.00000,NaN
8,NaN,NaN,NaN,NaN,0.00000,NaN
9,NaN,NaN,NaN,cost,250.00000,54.9
10,NaN,NaN,NaN,SP in THB,790.00000,150.0
11,NaN,NaN,NaN,Food cost %,31.64557,NaN


In [ ]:
list_products: list[dict] = []
current_product: dict = {}
list_ingredients: list[dict] = []

for index, row in df_test_hotel2.iterrows():
    # First item row
    if str(row[0]) != -1:
        current_product = {
            "name": row[0][11:]
        }

        continue
    
    # Last item row
    if str(row[0]) == 'Total for Stock Item':
        current_product['total'] : dict = {
            # TODO rename
            'total_first_col': row[11],
            'total_second_col': row[17],
            'total_third_col': str(row[21]),
            'total_fourth_col': row[24]
            
        }

        current_product['ingredients'] = list_ingredients
        list_products.append(current_product)
        
        list_ingredients = []

        continue
        
    # Intermediate row (Ingredient)
    list_ingredients.append({
        'code': row[0],
        'name': row[8],
        
        # TODO rename those columns
        'L': row[14],
        'S': row[20],
        'Qty': row[24],
        'Unit': row[32],
        'Sell_price': row[36],
        'Cost%': row[42]
    })


In [73]:
excel_file_path = 'data\\20231207_client_data\\recipe manual input on excel\\Hotel 2\\chef_manual_recipe_hotel2  .xlsx'
excel_file = pd.ExcelFile(excel_file_path)

# List to hold dataframes
dfs = []
list_products = []  # List to hold all products
current_product = {}  # Dictionary to hold the current product being processed
list_ingredients = []  # List to hold ingredients of the current product

# Iterate over all sheets
for sheet_name in excel_file.sheet_names:
    # Read each sheet into a dataframe
    df = pd.read_excel(
        excel_file_path,
        sheet_name=sheet_name,
        skiprows=21,
        header=None
    )
    df = df.dropna(axis = 0, how = 'all')
    df = df[~df[0].isin(['- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -'])]
    for index, row in df.iterrows():
        # First item row
        if pd.notnull(row[2]) or pd.notnull(row[3]) and str(row[0]).startswith('Template : '):
            if current_product.get("name"):  # Check if there is a current product being processed
                current_product['ingredients'] = list_ingredients
                list_products.append(current_product)
                list_ingredients = []  # Reset the list for the next product
            current_product = {
                "name": row[0][11:]
            }
            continue
        
        # Last item row
        if pd.notnull(row[0]) and str(row[0]) == 'Total for Stock Item':
            current_product['total'] = {
                'total_first_col': row[11],
                'total_second_col': row[17],
                'total_third_col': str(row[21]),
                'total_fourth_col': row[24]
            }
            current_product['ingredients'] = list_ingredients
            list_products.append(current_product)
            current_product = {}  # Reset current_product for the next product
            list_ingredients = []  # Reset the list for the next product
            continue
        
        # Intermediate row (Ingredient)
        code = row[0] if pd.notnull(row[0]) else '000'
        name = row[8] if pd.notnull(row[8]) else row[0]
    
        list_ingredients.append({
            'code': code,
            'name': name,
            'L': row[14] if pd.notnull(row[14]) else '000',
            'S': row[20] if pd.notnull(row[20]) else '000',
            'Qty': row[24] if pd.notnull(row[24]) else '000',
            'Unit': row[32] if pd.notnull(row[32]) else '000',
            'Sell_price': row[36] if pd.notnull(row[36]) else '000',
            'Cost%': row[42] if pd.notnull(row[42]) else '000'
        })

    # Append the dataframe to the list
    dfs.append(df)

# Concatenate all dataframes into one
merged_df = pd.concat(dfs, ignore_index=True)

merged_df

,0,1,2,3,4,5,6
0,NaN,NaN,Tiramisu,NaN,NaN,NaN,NaN
1,NaN,NaN,Ingredients:,NaN,NaN,NaN,NaN
2,NaN,432.827,Tiramisu,0.18,465,83.7,NaN
3,NaN,NaN,chocholate ball,0.01,2000,20.0,NaN
4,NaN,DZ0001,extra cost,1.0,5,5.0,NaN
...,...,...,...,...,...,...,...
126,NaN,DZ0001,extra cost,1.0,5,5.0,NaN
127,NaN,NaN,NaN,NaN,NaN,0.0,NaN
128,NaN,NaN,NaN,NaN,cost,32.684,NaN
129,NaN,NaN,NaN,NaN,SP in THB,150.0,NaN


In [ ]:
def convert_iscala_excel_to_json(file_path1,output_file_path1,file_path2,output_file_path2,file_path3,output_file_path3):
    """
    Converts three Excel files to JSON files

    :param file_path1: str, path to the first Excel file.
    :param file_path2: str, path to the second Excel file.
    :param file_path3: str, path to the third Excel file.
    :param output_file_path1: str, path where the first json file will be saved.
    :param output_file_path2: str, path where the second json file will be saved.
    :param output_file_path3: str, path where the third jsonfile will be saved.
    """
    # Creation of all the dictionnary for the different input file
    list_products: list[dict] = []
    current_product: dict = {}
    list_ingredients: list[dict] = []
    list_products2: list[dict] = []
    current_product2: dict = {}
    list_ingredients2: list[dict] = []
    list_products3: list[dict] = []
    current_product3: dict = {}
    list_ingredients3: list[dict] = []
    
    # Read all excel file from different source 
    # Drop all NaN row
    # Drop special row
    df: pd.DataFrame = pd.read_excel(file_path1,sheet_name='Sheet1',skiprows=6,header=None)
    df = df.dropna(axis = 0, how = 'all')
    df = df[~df[0].isin(['- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -'])]
    df2: pd.DataFrame = pd.read_excel(file_path2,sheet_name='Sheet1',skiprows=6,header=None)
    df2 = df2.dropna(axis = 0, how = 'all')
    df2 = df2[~df2[0].isin(['- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -'])]
    df3: pd.DataFrame = pd.read_excel(file_path2,sheet_name='Sheet1',skiprows=6,header=None)
    df3 = df3.dropna(axis = 0, how = 'all')
    df3 = df3[~df3[0].isin(['- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -'])]
   
   # File numero 1
    for row in df.iterrows():
        # get the name for the first item
        if pd.notnull(row[0]) and str(row[0]).startswith('Template : '):
            if current_product.get("name"):  
                current_product['ingredients'] = list_ingredients
                list_products.append(current_product)
                list_ingredients = []  
                current_product = {
                    "name": row[0][11:]
                }
                continue
        # get the sell price and cost with a special flag
        if pd.notnull(row[0]) and str(row[0]) == 'Total for Stock Item':
            current_product['total'] = {
                'Sell price': row[11],
                'Sell': row[17],
                'Cost': str(row[21]),
                'Cost%': row[24]
            }
            current_product['ingredients'] = list_ingredients
            list_products.append(current_product)
            current_product = {}  
            list_ingredients = []  
            continue
        
        # get all the ingredient and push it to the dictionnary 
        list_ingredients.append({
            'code': row[0] if pd.notnull(row[0]) else '000',
            'name': row[8] if pd.notnull(row[8]) else row[0],
            'L': row[14] if pd.notnull(row[14]) else '000',
            'S': row[20] if pd.notnull(row[20]) else '000',
            'Qty': row[24] if pd.notnull(row[24]) else '000',
            'Unit': row[32] if pd.notnull(row[32]) else '000',
            'Sell_price': row[36] if pd.notnull(row[36]) else '000',
            'Cost%': row[42] if pd.notnull(row[42]) else '000'
        })

        # verification for the double
        if current_product.get("name") and (not list_products or current_product != list_products[-1]):
            current_product['ingredients'] = list_ingredients
            list_products.append(current_product)
    
    # File Numéro 2
    for row in df2.iterrows():

        if pd.notnull(row[0]) and str(row[0]).startswith('Template : '):
            if current_product2.get("name"):  
                current_product2['ingredients'] = list_ingredients2
                list_products2.append(current_product2)
                list_ingredients2 = []  
                current_product2 = {
                    "name": row[0][11:]
                }
                continue
    
        if pd.notnull(row[0]) and str(row[0]) == 'Total for Stock Item':
            current_product2['total'] = {
                'Sell price': row[11],
                'Sell': row[17],
                'Cost': str(row[21]),
                'Cost%': row[24]
            }
            current_product2['ingredients'] = list_ingredients2
            list_products2.append(current_product2)
            current_product2 = {}  
            list_ingredients2 = []  
            continue
    
        list_ingredients2.append({
            'code': row[0] if pd.notnull(row[0]) else '000',
            'name': row[8] if pd.notnull(row[8]) else row[0],
            'L': row[14] if pd.notnull(row[14]) else '000',
            'S': row[20] if pd.notnull(row[20]) else '000',
            'Qty': row[24] if pd.notnull(row[24]) else '000',
            'Unit': row[32] if pd.notnull(row[32]) else '000',
            'Sell_price': row[36] if pd.notnull(row[36]) else '000',
            'Cost%': row[42] if pd.notnull(row[42]) else '000'
        })


        if current_product2.get("name") and (not list_products2 or current_product2 != list_products2[-1]):
            current_product2['ingredients'] = list_ingredients2
            list_products2.append(current_product2)
    
    # File numéro 3
    for row in df3.iterrows():

        if pd.notnull(row[0]) and str(row[0]).startswith('Template : '):
            if current_product3.get("name"):  
                current_product3['ingredients'] = list_ingredients3
                list_products3.append(current_product3)
                list_ingredients3 = []  
                current_product3 = {
                    "name": row[0][11:]
                }
                continue
    
        if pd.notnull(row[0]) and str(row[0]) == 'Total for Stock Item':
            current_product3['total'] = {
                'Sell price': row[11],
                'Sell': row[17],
                'Cost': str(row[21]),
                'Cost%': row[24]
            }
            current_product3['ingredients'] = list_ingredients3
            list_products3.append(current_product3)
            current_product3 = {}  
            list_ingredients3 = []  
            continue
    
        list_ingredients3.append({
            'code': row[0] if pd.notnull(row[0]) else '000',
            'name': row[8] if pd.notnull(row[8]) else row[0],
            'L': row[14] if pd.notnull(row[14]) else '000',
            'S': row[20] if pd.notnull(row[20]) else '000',
            'Qty': row[24] if pd.notnull(row[24]) else '000',
            'Unit': row[32] if pd.notnull(row[32]) else '000',
            'Sell_price': row[36] if pd.notnull(row[36]) else '000',
            'Cost%': row[42] if pd.notnull(row[42]) else '000'
        })


        if current_product3.get("name") and (not list_products3 or current_product3 != list_products3[-1]):
            current_product3['ingredients'] = list_ingredients3
            list_products3.append(current_product3)

    # create the output file for the json format
    with open(output_file_path1, "w") as outfile:
        json.dump(list_products, outfile, indent=4)

    with open(output_file_path2, "w") as outfile:
        json.dump(list_products2, outfile, indent=4)

    with open(output_file_path3, "w") as outfile:
        json.dump(list_products3, outfile, indent=4)

    return f"Files converted and saved as '{output_file_path1}' and '{output_file_path2}' and '{output_file_path3}'"